In [1]:
from pyspark.sql import SparkSession
from pysparkling import *
from pyspark.sql.types import *

ss = SparkSession.builder.getOrCreate()
hc = H2OContext.getOrCreate()

22/03/02 13:50:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/02 13:50:40 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/03/02 13:50:40 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/03/02 13:50:40 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/03/02 13:50:40 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
22/03/02 13:50:41 WARN InternalH2OBackend: Increasing 'spark.locality.wait' to value 0 (Infinitive) as we need to ensure we run on the nodes with H2O


03-02 13:50:43.775 192.168.0.166:54325   82857    Thread-4  INFO water.default: ----- H2O started  -----
03-02 13:50:43.776 192.168.0.166:54325   82857    Thread-4  INFO water.default: Build git branch: rel-zorn
03-02 13:50:43.776 192.168.0.166:54325   82857    Thread-4  INFO water.default: Build git hash: 717d8bf831d5d6b0decda9c37a2a20de9a491754
03-02 13:50:43.776 192.168.0.166:54325   82857    Thread-4  INFO water.default: Build git describe: jenkins-3.36.0.2-53-g717d8bf
03-02 13:50:43.776 192.168.0.166:54325   82857    Thread-4  INFO water.default: Build project version: 3.36.0.3
03-02 13:50:43.777 192.168.0.166:54325   82857    Thread-4  INFO water.default: Build age: 13 days
03-02 13:50:43.777 192.168.0.166:54325   82857    Thread-4  INFO water.default: Built by: 'jenkins'
03-02 13:50:43.777 192.168.0.166:54325   82857    Thread-4  INFO water.default: Built on: '2022-02-16 17:51:32'
03-02 13:50:43.777 192.168.0.166:54325   82857    Thread-4  INFO water.default: Found H2O Core exte

H2O_cluster_uptime:,10 secs
H2O_cluster_timezone:,Pacific/Pitcairn
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.0.3
H2O_cluster_version_age:,13 days
H2O_cluster_name:,sparkling-water-dwoodbridge_local-1646257840286
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,798 Mb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"



Sparkling Water Context:
 * Sparkling Water Version: 3.36.0.3-1-3.2
 * H2O name: sparkling-water-dwoodbridge_local-1646257840286
 * cluster size: 1
 * list of used nodes:
  (executorId, host, port)
  ------------------------
  (0,192.168.0.166,54325)
  ------------------------

  Open H2O Flow in browser: http://192.168.0.166:54324 (CMD + click in Mac OSX)

    


## Create from Spark DataFrame

In [2]:
def toDoubleSafe(v):
    try:
        return float(v)
    except:
        return str(v) #if it is not a float type return as a string.
    
adultschema = StructType([
    StructField("age",DoubleType(),True),
    StructField("workclass",StringType(),True),
    StructField("fnlwgt",DoubleType(),True),
    StructField("education",StringType(),True),
    StructField("marital_status",StringType(),True),
    StructField("occupation",StringType(),True),
    StructField("relationship",StringType(),True),
    StructField("race",StringType(),True),
    StructField("sex",StringType(),True),
    StructField("capital_gain",DoubleType(),True),
    StructField("capital_loss",DoubleType(),True),
    StructField("hours_per_week",DoubleType(),True),
    StructField("native_country",StringType(),True),
    StructField("income",StringType(),True)
])

In [3]:
#load and convert the data
census_raw = ss.sparkContext.textFile("../Data/adult.raw", 4).map(lambda x:  x.split(", "))
census_raw = census_raw.map(lambda row:  [toDoubleSafe(x) for x in row])

dfraw = ss.createDataFrame(census_raw, adultschema)


03-02 13:50:53.004 192.168.0.166:54325   82857    Thread-4  INFO org.apache.spark.storage.memory.MemoryStore: Block broadcast_0 stored as values in memory (estimated size 355.1 KiB, free 366.0 MiB)
03-02 13:50:53.133 192.168.0.166:54325   82857    Thread-4  INFO org.apache.spark.storage.memory.MemoryStore: Block broadcast_0_piece0 stored as bytes in memory (estimated size 32.2 KiB, free 365.9 MiB)
03-02 13:50:53.135 192.168.0.166:54325   82857  agerMaster  INFO org.apache.spark.storage.BlockManagerInfo: Added broadcast_0_piece0 in memory on 192.168.0.166:49466 (size: 32.2 KiB, free: 366.3 MiB)
03-02 13:50:53.138 192.168.0.166:54325   82857    Thread-4  INFO org.apache.spark.SparkContext: Created broadcast 0 from textFile at NativeMethodAccessorImpl.java:0
03-02 13:50:53.342 192.168.0.166:54325   82857    Thread-4  INFO org.apache.spark.sql.internal.SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
03-02 13:50:53.343 192.168.0.166:54325 

In [4]:
adult_h2o = hc.asH2OFrame(dfraw, "adult")

03-02 13:50:55.005 192.168.0.166:54325   82857    Thread-4  INFO org.apache.spark.sql.catalyst.expressions.codegen.CodeGenerator: Code generated in 193.958105 ms
03-02 13:50:55.097 192.168.0.166:54325   82857  8298793-92  INFO water.default: POST /3/InitializeFrame, parms: {key=adult, columns=["age","workclass","fnlwgt","education","marital_status","occupation","relationship","race","sex","capital_gain","capital_loss","hours_per_week","native_country","income"]}
03-02 13:50:55.122 192.168.0.166:54325   82857    Thread-4  INFO ai.h2o.sparkling.H2OFrame: H2O node http://192.168.0.166:54325/3/InitializeFrame successfully responded for the POST.
03-02 13:50:55.249 192.168.0.166:54325   82857    Thread-4  INFO org.apache.spark.SparkContext: Starting job: collect at Writer.scala:169
03-02 13:50:55.263 192.168.0.166:54325   82857  event-loop  INFO org.apache.spark.scheduler.DAGScheduler: Got job 0 (collect at Writer.scala:169) with 4 output partitions
03-02 13:50:55.263 192.168.0.166:54325   

03-02 13:50:56.272 192.168.0.166:54325   82857  .0 (TID 3)  INFO org.apache.spark.rdd.HadoopRDD: Input split: file:/Users/dwoodbridge/Class/2022_MSDS697/Examples/Data/adult.raw:4340982+1446996
03-02 13:50:56.272 192.168.0.166:54325   82857  .0 (TID 0)  INFO org.apache.spark.rdd.HadoopRDD: Input split: file:/Users/dwoodbridge/Class/2022_MSDS697/Examples/Data/adult.raw:0+1446994
03-02 13:50:56.272 192.168.0.166:54325   82857  .0 (TID 2)  INFO org.apache.spark.rdd.HadoopRDD: Input split: file:/Users/dwoodbridge/Class/2022_MSDS697/Examples/Data/adult.raw:2893988+1446994
03-02 13:50:56.272 192.168.0.166:54325   82857  .0 (TID 1)  INFO org.apache.spark.rdd.HadoopRDD: Input split: file:/Users/dwoodbridge/Class/2022_MSDS697/Examples/Data/adult.raw:1446994+1446994
03-02 13:50:57.336 192.168.0.166:54325   82857  .0 (TID 2)  INFO org.apache.spark.sql.catalyst.expressions.codegen.CodeGenerator: Code generated in 36.729605 ms
03-02 13:50:57.655 192.168.0.166:54325   82857  .0 (TID 3)  INFO org.apac

03-02 13:50:58.307 192.168.0.166:54325   82857  .0 (TID 5)  INFO org.apache.spark.api.python.PythonRunner: Times: total = 565, boot = -61, init = 76, finish = 550
03-02 13:50:58.316 192.168.0.166:54325   82857  .0 (TID 6)  INFO org.apache.spark.api.python.PythonRunner: Times: total = 573, boot = -63, init = 76, finish = 560
03-02 13:50:58.320 192.168.0.166:54325   82857  .0 (TID 4)  INFO org.apache.spark.api.python.PythonRunner: Times: total = 579, boot = -61, init = 64, finish = 576
03-02 13:50:58.324 192.168.0.166:54325   82857  .0 (TID 7)  INFO org.apache.spark.api.python.PythonRunner: Times: total = 577, boot = -57, init = 71, finish = 563
03-02 13:50:58.409 192.168.0.166:54325   82857  .0 (TID 5)  INFO ai.h2o.sparkling.backend.H2OChunk: H2O node http://192.168.0.166:54325/3/Chunk?frame_name=adult&num_rows=12201&compression=SNAPPY&maximum_vector_sizes=&expected_types=BwsHCwsLCwsLBwcHCws%3D&chunk_id=1 successfully responded for the PUT.
03-02 13:50:58.412 192.168.0.166:54325   82857

In [5]:
adult_h2o.show()

age,workclass,fnlwgt,education,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
39,State-gov,77516,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
50,Self-emp-not-inc,83311,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
38,Private,215646,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
53,Private,234721,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
28,Private,338409,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
37,Private,284582,Masters,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
49,Private,160187,9th,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K
52,Self-emp-not-inc,209642,HS-grad,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K
31,Private,45781,Masters,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,>50K
42,Private,159449,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,>50K


In [6]:
adult_h2o.summary()

,age,workclass,fnlwgt,education,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
type,int,enum,int,enum,enum,enum,enum,enum,enum,int,int,int,enum,enum
mins,17.0,,12285.0,,,,,,,0.0,0.0,1.0,,
mean,38.64358543876177,,189664.13459727296,,,,,,,1079.0676262233324,87.50231358257236,40.42238237582413,,
maxs,90.0,,1490400.0,,,,,,,99999.0,4356.0,99.0,,
sigma,13.710509934443552,,105604.02542315725,,,,,,,7452.019057655393,403.00455212435895,12.391444024252296,,
zeros,0,,0,,,,,,,44807,46560,0,,
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,39.0,State-gov,77516.0,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40.0,United-States,<=50K
1,50.0,Self-emp-not-inc,83311.0,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States,<=50K
2,38.0,Private,215646.0,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K


## Create from File

In [8]:
import h2o
adult_h2o = h2o.import_file(path="../Data/adult.raw")

03-02 13:51:00.229 192.168.0.166:54325   82857  298793-129  INFO water.default: POST /3/ImportFilesMulti, parms: {paths=[/Users/dwoodbridge/Class/2022_MSDS697/Examples/Data/adult.raw]}
03-02 13:51:00.248 192.168.0.166:54325   82857  298793-101  INFO water.default: POST /3/ParseSetup, parms: {single_quotes=False, source_frames=["nfs://Users/dwoodbridge/Class/2022_MSDS697/Examples/Data/adult.raw"], check_header=0}
03-02 13:51:00.339 192.168.0.166:54325   82857  agerMaster  INFO org.apache.spark.storage.BlockManagerInfo: Removed broadcast_2_piece0 on 192.168.0.166:49466 in memory (size: 16.8 KiB, free: 366.3 MiB)
03-02 13:51:00.703 192.168.0.166:54325   82857  298793-101  INFO water.default: ParseSetup heuristic: cloudSize: 1, cores: 16, numCols: 14, maxLineLength: 150, totalSize: 5787978, localParseSize: 5787978, chunkSize: 90438, numChunks: 63, numChunks * cols: 882
03-02 13:51:00.721 192.168.0.166:54325   82857  298793-129  INFO water.default: POST /3/Parse, parms: {number_columns=14, 

In [9]:
adult_h2o.summary()

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14
type,int,enum,int,enum,enum,enum,enum,enum,enum,int,int,int,enum,enum
mins,17.0,,12285.0,,,,,,,0.0,0.0,1.0,,
mean,38.64358543876172,,189664.13459727284,,,,,,,1079.0676262233321,87.50231358257237,40.422382375824085,,
maxs,90.0,,1490400.0,,,,,,,99999.0,4356.0,99.0,,
sigma,13.710509934443557,,105604.02542315728,,,,,,,7452.019057655394,403.004552124359,12.391444024252307,,
zeros,0,,0,,,,,,,44807,46560,0,,
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,39.0,State-gov,77516.0,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40.0,United-States,<=50K
1,50.0,Self-emp-not-inc,83311.0,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States,<=50K
2,38.0,Private,215646.0,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K


In [10]:
adult_h2o.set_names(["age", "capital_gain", "capital_loss", "education", "fnlwgt", "hours_per_week", "income", "marital_status", "native_country", "occupation", "race", "relationship", "sex", "workclass"])

03-02 13:51:01.430 192.168.0.166:54325   82857  298793-129  INFO water.default: POST /4/sessions, parms: {}
03-02 13:51:01.438 192.168.0.166:54325   82857  298793-101  INFO water.default: POST /99/Rapids, parms: {ast=(tmp= py_1_sid_a2a8 (colnames= adult_raw.hex [0 1 2 3 4 5 6 7 8 9 10 11 12 13] ['age' 'capital_gain' 'capital_loss' 'education' 'fnlwgt' 'hours_per_week' 'income' 'marital_status' 'native_country' 'occupation' 'race' 'relationship' 'sex' 'workclass'])), session_id=_sid_a2a8}
03-02 13:51:01.446 192.168.0.166:54325   82857  298793-129  INFO water.default: GET /3/Frames/py_1_sid_a2a8, parms: {column_offset=0, full_column_count=-1, row_count=10, row_offset=0, column_count=-1}


age,capital_gain,capital_loss,education,fnlwgt,hours_per_week,income,marital_status,native_country,occupation,race,relationship,sex,workclass
39,State-gov,77516,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
50,Self-emp-not-inc,83311,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
38,Private,215646,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
53,Private,234721,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
28,Private,338409,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
37,Private,284582,Masters,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
49,Private,160187,9th,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K
52,Self-emp-not-inc,209642,HS-grad,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K
31,Private,45781,Masters,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,>50K
42,Private,159449,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,>50K


## Convert as Spark DataFrame

In [11]:
adult_df = hc.asSparkFrame(adult_h2o)

03-02 13:51:01.463 192.168.0.166:54325   82857  8298793-83  INFO water.default: GET /3/Frames/py_1_sid_a2a8/summary, parms: {row_count=0}
03-02 13:51:01.479 192.168.0.166:54325   82857    Thread-4  INFO ai.h2o.sparkling.H2OFrame: H2O node http://192.168.0.166:54325/3/Frames/py_1_sid_a2a8/summary?row_count=0 successfully responded for the GET.
03-02 13:51:01.486 192.168.0.166:54325   82857    Thread-4  INFO ai.h2o.sparkling.backend.utils.RestApiUtils: H2O node http://192.168.0.166:54325/3/Cloud successfully responded for the GET.
03-02 13:51:01.488 192.168.0.166:54325   82857  8298793-83  INFO water.default: GET /3/FrameChunks/py_1_sid_a2a8, parms: {}
03-02 13:51:01.489 192.168.0.166:54325   82857    Thread-4  INFO ai.h2o.sparkling.H2OFrame: H2O node http://192.168.0.166:54325/3/FrameChunks/py_1_sid_a2a8 successfully responded for the GET.


In [12]:
adult_df.show()

03-02 13:51:01.674 192.168.0.166:54325   82857    Thread-4  INFO org.apache.spark.sql.catalyst.expressions.codegen.CodeGenerator: Code generated in 14.452289 ms
03-02 13:51:01.682 192.168.0.166:54325   82857    Thread-4  INFO org.apache.spark.SparkContext: Starting job: showString at NativeMethodAccessorImpl.java:0
03-02 13:51:01.682 192.168.0.166:54325   82857  event-loop  INFO org.apache.spark.scheduler.DAGScheduler: Got job 2 (showString at NativeMethodAccessorImpl.java:0) with 1 output partitions
03-02 13:51:01.682 192.168.0.166:54325   82857  event-loop  INFO org.apache.spark.scheduler.DAGScheduler: Final stage: ResultStage 2 (showString at NativeMethodAccessorImpl.java:0)
03-02 13:51:01.682 192.168.0.166:54325   82857  event-loop  INFO org.apache.spark.scheduler.DAGScheduler: Parents of final stage: List()
03-02 13:51:01.683 192.168.0.166:54325   82857  event-loop  INFO org.apache.spark.scheduler.DAGScheduler: Missing parents: List()
03-02 13:51:01.684 192.168.0.166:54325   82857

In [13]:
ss.stop()

03-02 13:51:01.829 192.168.0.166:54325   82857    Thread-4  INFO org.apache.spark.ui.SparkUI: Stopped Spark web UI at http://192.168.0.166:4044
03-02 13:51:01.841 192.168.0.166:54325   82857  nt-loop-10  INFO org.apache.spark.MapOutputTrackerMasterEndpoint: MapOutputTrackerMasterEndpoint stopped!
03-02 13:51:01.860 192.168.0.166:54325   82857    Thread-4  INFO org.apache.spark.storage.memory.MemoryStore: MemoryStore cleared
03-02 13:51:01.860 192.168.0.166:54325   82857    Thread-4  INFO org.apache.spark.storage.BlockManager: BlockManager stopped
03-02 13:51:01.862 192.168.0.166:54325   82857    Thread-4  INFO org.apache.spark.storage.BlockManagerMaster: BlockManagerMaster stopped
03-02 13:51:01.865 192.168.0.166:54325   82857  nt-loop-14  INFO org.apache.spark.scheduler.OutputCommitCoordinator$OutputCommitCoordinatorEndpoint: OutputCommitCoordinator stopped!
03-02 13:51:01.875 192.168.0.166:54325   82857    Thread-4  INFO org.apache.spark.SparkContext: Successfully stopped SparkContex